### Triple graph to cypher
You can use this code to transform graphs as concatenated triples, with the answer triple at the start, to a cypher CREATE query. This way it can be loaded into Neo4j.

In [3]:
import re

In [4]:
def var_name(string, number):
    """ Convert node name to valid and unique Cypher variable name. """
    s = re.sub(r'[^\w\s]','',string)
    result = '_'.join([e.lower() for e in s.split()])
    if result[0].isnumeric():
        result = 'a' + result
    return result + str(number)

def get_nodes(elements):
    nodes = []
    for i in range(len(elements)-1):
        if i%3 == 1:
            continue
        nodes.append(elements[i+1])
    return list(set(nodes))

def convert(graph, number): 
    # get all elements
    elements = re.split(r'<[A H R T]> ', graph)[1:]
    elements = [e.strip() for e in elements]
    
    # get nodes
    nodes = get_nodes(elements)
    
    # add answer node
    answer = elements[0]
    if answer == '':
        result = "CREATE "
    else:
        result = f"CREATE ({var_name(answer, number)}:answer {{name:\"{answer}\", q:{number}}}), "    
    
    # add other nodes
    for node in nodes:
        if node.lower() == answer:
            continue
        result += f"({var_name(node, number)}:normal {{name:\"{node}\", q:{number}}}), "
    
    # add relations
    for i in range(int((len(elements)-1)/3)):
        h = elements[i*3+1]
        r = elements[i*3+2]
        t = elements[i*3+3]
        
        result += f"({var_name(h, number)})-[:{var_name(r, '')}]->({var_name(t, number)}), "
    
    return result[:-2]

In [3]:
in_file = 'data/PQ/src_a_test.txt'
with open(in_file, 'r',encoding='utf-8') as f:
    data = f.read().splitlines()

In [4]:
out_file = 'cypher_graphsPQ.txt'
with open(out_file, 'w',encoding='utf-8') as f:
    for i, graph in enumerate(data):
        f.write(convert(graph, i+2000) + '\n')